In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV

df = pd.read_csv("mnist.csv")
X = df.drop(columns=["class"]).to_numpy()
Y = df["class"].to_numpy()

# no hay que normalizar
# no hay que codificar
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, stratify=Y)

In [23]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
params = {
    "C": (1, 2, 10),
    "max_iter": (100,)
}
svm_linear_cv = GridSearchCV(svc, params, cv=3)
svm_linear_cv.fit(X_train, Y_train)
svm_linear_cv.cv_results_

/usr/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, i

{'mean_fit_time': array([6.31437739, 6.58169993, 6.81753198]),
 'std_fit_time': array([0.0748377 , 0.0493917 , 0.08963598]),
 'mean_score_time': array([0.08396832, 0.07695611, 0.07754342]),
 'std_score_time': array([0.00973741, 0.00125373, 0.00097912]),
 'param_C': masked_array(data=[1, 2, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_iter': masked_array(data=[100, 100, 100],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'max_iter': 100},
  {'C': 2, 'max_iter': 100},
  {'C': 10, 'max_iter': 100}],
 'split0_test_score': array([0.89391204, 0.887268  , 0.8767242 ]),
 'split1_test_score': array([0.89687297, 0.89687297, 0.86639705]),
 'split2_test_score': array([0.8848765 , 0.89563773, 0.85981511]),
 'mean_test_score': array([0.89188717, 0.89325957, 0.86764545]),
 'std_test_score': array([0.00510254, 0.00426658, 0.00695932]),
 'rank_test_score': array([2, 1, 

In [24]:
from joblib import dump

final_svc = LinearSVC(C=1, max_iter=1000)
final_svc.fit(X, Y)
dump(final_svc, "svm_linear.joblib")

/usr/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


['svm_linear.joblib']